# Initiate
We first initiate the environment. **Use python 3.9 kenrel if running from jupyterlab**

In [1]:
import os
import bentoml
import mlflow
import darts 
#print(numpy.__version__)

from dotenv import load_dotenv
load_dotenv()


True

# Load model as bentos

Then we load the model from minio as a bento. In the link to the model we always use the pyfunc model folder. The bento name needs to be changed if a different model needs to be loaded, otherwise the new model will be considered a newer version of the old one.

In [4]:
model = bentoml.mlflow.import_model(
    "uc7_companies_no_covs", #bento model name 
    "s3://mlflow-bucket/40/2850a79365524979819f69e9bbd55514/artifacts/pyfunc_model", #minio uri
    signatures={'predict': {'batchable': True}})

bento_model = bentoml.mlflow.get(model.tag)
pyfunc_model: mlflow.pyfunc.PyFuncModel = bentoml.mlflow.load_model(
    model.tag)

2024/01/18 00:33:03 WARNING mlflow.pyfunc: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - torch (current: 2.1.2, required: torch==1.11.0+cu102)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.


Local path inside _load_pyfunc: /home/theo/bentoml/models/uc7_companies_no_covs/l7ddjq5vrcr4eabl/mlflow_model/data/2850a79365524979819f69e9bbd55514
Local path altered for loading: /home/theo/bentoml/models/uc7_companies_no_covs/l7ddjq5vrcr4eabl/mlflow_model/data/2850a79365524979819f69e9bbd55514
/home/theo/bentoml/models/uc7_companies_no_covs/l7ddjq5vrcr4eabl/mlflow_model/data/2850a79365524979819f69e9bbd55514
/home/theo/bentoml/models/uc7_companies_no_covs/l7ddjq5vrcr4eabl/mlflow_model/data/2850a79365524979819f69e9bbd55514

Loading local PKL model...


/home/theo/anaconda3/envs/test2/lib/python3.9/site-packages/statsforecast/core.py:26: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm
/home/theo/anaconda3/envs/test2/lib/python3.9/site-packages/sklearn/base.py:348: InconsistentVersionWarning: Trying to unpickle estimator MinMaxScaler from version 1.0.2 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


Then we make sure that the model has been saved as a bento, and we get the tag of its latest version

In [2]:
print(bentoml.models.list("uc7_companies_no_covs")[0])

Model(tag="uc7_companies_no_covs:hurwbr5vq6ituabl")


# Predict

To make sure that this model is providing predictions normaly, we test it using the series that is uploaded to minio

## LGBM

In [6]:
%%capture
# input example
input_example = {
            "n": "24",
            "series_uri": "s3://mlflow-bucket/40/b92fb3c675814fdebf08c23c7de9c001/artifacts/features/series.csv",
            "roll_size": "1",
            "future_covariates_uri": "None",
            "past_covariates_uri": "None",
            "batch_size": "1",
            "multiple": "True",
            "weather_covariates":[],
            "resolution": "60",
            "ts_id_pred": "BBB6030",
}
predictions = pyfunc_model.predict(input_example)

In [7]:
predictions

component,Value
Datetime,
2023-12-28 00:00:00,3.931897
2023-12-28 01:00:00,3.949497
2023-12-28 02:00:00,3.980993
2023-12-28 03:00:00,4.072373
2023-12-28 04:00:00,4.106251
2023-12-28 05:00:00,4.013561
2023-12-28 06:00:00,3.740512
2023-12-28 07:00:00,3.039017
2023-12-28 08:00:00,1.821295
